# Forecastability

### Loading Libraries

In [ ]:
%cd ../..

In [ ]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd

# Data Visualization
import seaborn as sns
import plotly.io as pio
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Warnings
import warnings
import humanize

# IO & Requests
import time
import random
import requests
from io import StringIO

# StatsModels
import statsmodels.api as sm
from statsmodels.tsa.seasonal import MSTL , DecomposeResult

# OS
import os
import sys
import pickleshare
import missingno as msno
from itertools import cycle

# PyArrow
import pyarrow as pa

# FuncTools
from functools import partial

# Path & Notebook Optimizer
from pathlib import Path
import missingno as msno
from tqdm.auto import tqdm

# Scikit-Learn
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

# IPython
from IPython.display import display, HTML

# NIXTLA
from statsforecast.core import StatsForecast
from utilsforecast.plotting import plot_series
from utilsforecast.evaluation import evaluate

# Forecast
# from datasetsforecast.losses import *
from utilsforecast.evaluation import evaluate
from statsforecast.losses import mse, mae, mape, rmse, smape, mase

# Custom Libraries
from src.utils import plotting_utils
from src.utils.ts_utils import forecast_bias

In [ ]:
#from darts.models import Theta
#from darts.utils.utils import ModelMode, SeasonalityMode

In [ ]:
!pip install statsforecast

In [ ]:
tqdm.pandas()

random.seed(42)

np.random.seed(42)

pio.templates.default = "plotly_white"

sys.path.append('/Users/joaquinromero/Desktop/MTSF')

In [ ]:
warnings.filterwarnings("ignore", category=UserWarning)

warnings.filterwarnings("ignore", category=FutureWarning)

warnings.filterwarnings("ignore", message="'force_all_finite' was renamed to 'ensure_all_finite'")

In [ ]:
if 'NIXTLA_ID_AS_COL' in os.environ:
    del os.environ['NIXTLA_ID_AS_COL']
    
os.environ['NIXTLA_ID_AS_COL'] = '1'

In [ ]:
os.makedirs("imgs/chapter_04", exist_ok=True)
preprocessed = Path.home() / "Desktop" / "data" / "london_smart_meters" / "preprocessed"

output = Path("data/london_smart_meters/output")

In [ ]:
def format_plot(fig, legends = None, xlabel="Time", ylabel="Value"):
    if legends:
        names = cycle(legends)
        fig.for_each_trace(lambda t:  t.update(name = next(names)))
    fig.update_layout(
            autosize=False,
            width=900,
            height=500,
            title={
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            titlefont={
                "size": 20
            },
            legend_title = None,
            yaxis=dict(
                title_text=ylabel,
                titlefont=dict(size=12),
            ),
            xaxis=dict(
                title_text=xlabel,
                titlefont=dict(size=12),
            )
        )
    return fig

## Reading & Selecting Households

In [ ]:
try:
    lclid_acorn_map = pd.read_pickle("data/london_smart_meters/preprocessed/london_smart_meters_lclid_acorn_map.pkl")
except FileNotFoundError:
    display(HTML("""
    <div class="alert alert-block alert-warning">
    <b>Warning!</b> File not found. Please make sure you have run 02 - Preprocessing London Smart Meter Dataset.ipynb in Chapter02
    </div>
    """))

In [ ]:
affluent_households = lclid_acorn_map.loc[lclid_acorn_map.Acorn_grouped=="Affluent", ["LCLid",'file']]

adversity_households = lclid_acorn_map.loc[lclid_acorn_map.Acorn_grouped=="Adversity", ["LCLid",'file']]

comfortable_households = lclid_acorn_map.loc[lclid_acorn_map.Acorn_grouped=="Comfortable", ["LCLid",'file']]

In [ ]:
size = 50

selected_households = pd.concat(
    [
        affluent_households.sample(size, random_state=76),
        comfortable_households.sample(size, random_state=76),
        adversity_households.sample(size, random_state=76),
    ]
)
selected_households['block']=selected_households.file.str.split("_", expand=True).iloc[:,1].astype(int)

In [ ]:
# Extracting The Paths to The Different Blocks and Extracting The Starting and Ending Blocks
path_blocks = [
    (p, *list(map(int, p.name.split("_")[5].split(".")[0].split("-"))))
    for p in Path("data/london_smart_meters/preprocessed").glob(
        "london_smart_meters_merged_block*"
    )
]

In [ ]:
household_df_l = []

for path, start_b, end_b in tqdm(path_blocks):
    block_df = pd.read_parquet(path)
    selected_households['block'].between
    mask = selected_households['block'].between(start_b, end_b)
    lclids = selected_households.loc[mask, "LCLid"]
    household_df_l.append(block_df.loc[block_df.LCLid.isin(lclids)])

In [ ]:
block_df = pd.concat(household_df_l)

del household_df_l
block_df.head()

### Filling in Missing Values

In [ ]:
from src.imputation.interpolation import SeasonalInterpolation

In [ ]:
block_df.energy_consumption = block_df.energy_consumption.progress_apply(lambda x: SeasonalInterpolation(seasonal_period=48*7).fit_transform(x.reshape(-1,1)).squeeze())

## Deseasonalize and Detrend

In [ ]:
from src.decomposition.seasonal import MultiSeasonalDecomposition

In [ ]:
def make_stationary(row):
#     print(row)
    # Order of row: LCLid, timestamp, frequency, energy_consumption
    ts = row[3]
    dates = pd.date_range(start=row[1], freq=row[2], periods=len(ts))
    stl = MultiSeasonalDecomposition(seasonal_model="fourier",seasonality_periods=["day_of_year", "day_of_week", "hour"], model = "additive", n_fourier_terms=10)
    res = stl.fit(pd.Series(ts, index=dates))
    return res.resid.values# + res.trend.values

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=DeprecationWarning) 
    block_df['residuals'] =[make_stationary(r) for r in tqdm(zip(*block_df.to_dict("list").values()), total=len(block_df))]

## Forecastability Metrics

### Coefficient of Variation

In [ ]:
from src.forecastability.cov import calc_cov

In [ ]:
block_df["cov"] = block_df.progress_apply(lambda x: calc_cov(x['energy_consumption']), axis=1)

In [ ]:
fig = px.histogram(x=block_df["cov"], title="Distribution of Coefficient of Variation")
fig = format_plot(fig, xlabel="Coefficient of Variation", ylabel="")
# fig.write_image("imgs/chapter_4/cov.png")
fig.show()

### Residual Variability

In [ ]:
from src.forecastability.cov import calc_norm_sd

In [ ]:
block_df["residual_variability"] = block_df.progress_apply(lambda x: calc_norm_sd(x['residuals'],x['energy_consumption']), axis=1)

In [ ]:
fig = px.histogram(x=block_df["residual_variability"], title="Distribution of Residual Variability")
fig = format_plot(fig, xlabel="Residual Variability", ylabel="")
# fig.write_image("imgs/chapter_4/rv.png")
fig.show()

### Spectral Entropy

In [ ]:
from src.forecastability.entropy import spectral_entropy

In [ ]:
%%time
block_df["residual_spectral_entropy"] = block_df.residuals.progress_apply(spectral_entropy)

In [ ]:
%%time
block_df["spectral_entropy"] = block_df.energy_consumption.progress_apply(lambda x: spectral_entropy(x, transform_stationary=True))

In [ ]:
fig = px.histogram(x=block_df["spectral_entropy"], title="Distribution of Spectral Entropy")
fig = format_plot(fig, xlabel="Spectral Entropy", ylabel="")
# fig.write_image("imgs/chapter_4/spectral_entropy.png")
fig.show()

In [ ]:
fig = px.histogram(x=block_df["residual_spectral_entropy"], title="Distribution of Residual Spectral Entropy")
fig = format_plot(fig, xlabel="Residual Spectral Entropy", ylabel="")
# fig.write_image("imgs/chapter_4/resid_spectral_entropy.png")
fig.show()

### Kaboudan Metric

In [ ]:
#cd ../../

In [ ]:
from src.forecastability.kaboudan import kaboudan_metric, modified_kaboudan_metric 

In [ ]:
%%time
block_size = 5
freq = '30min'
#models = [SeasonalNaive(season_length=48*7)]
#models = [SimpleExponentialSmoothing(alpha = 0.4, alias = 'SimpleExponentialSmoothing')]
models = [Theta(season_length=48*7, decomposition_type='additive')]


  
block_df["kaboudan_metric"] = [kaboudan_metric(r[0], 
                                               model=models[0], 
                                               block_size=block_size,  
                                               backtesting_start=0.5, 
                                               n_folds=1,
                                               freq = freq) 
                                               for r in tqdm(zip(*block_df[["energy_consumption"]].to_dict("list").values()), total=len(block_df))]

block_df[['LCLid','kaboudan_metric']].head()

In [ ]:
%%time
block_df["modified_kaboudan_metric"] = [modified_kaboudan_metric(r[0], 
                                               model=models[0], 
                                               block_size=block_size,  
                                               backtesting_start=0.5, 
                                               n_folds=1,
                                               freq = freq) 
                                               for r in tqdm(zip(*block_df[["energy_consumption"]].to_dict("list").values()), total=len(block_df))]

block_df[['LCLid','modified_kaboudan_metric']].head()

In [ ]:
fig = px.histogram(x=block_df["modified_kaboudan_metric"], title="Distribution of Modified Kaboudan Metric")
fig = format_plot(fig, xlabel="Modified Kaboudan Metric", ylabel="")
# fig.write_image("imgs/chapter_4/kaboudan_metric.png")
fig.show()

## Analysis

In [ ]:
forecastability_df = block_df[["LCLid",'spectral_entropy', 'residual_spectral_entropy',
       'modified_kaboudan_metric', 'cov', "kaboudan_metric",
       'residual_variability']]
rename_dict = {
    "spectral_entropy": "Spectral Entropy",
    "residual_spectral_entropy" : "Residual Spectral Entropy",
    "modified_kaboudan_metric": "Modified Kaboudan Metric",
    "kaboudan_metric": "Kaboudan Metric",
    "cov": "Coefficient of Variation",
    "residual_variability": "Residual Variability",
    
}
forecastability_df.to_pickle(output/"forecastability_metrics.pkl")

In [ ]:
def biplot(item_metrics_df, features, title="Loading Plot"):
    X = item_metrics_df[features].dropna()
    scaler = StandardScaler()
    scaler.fit(X)
    X=scaler.transform(X)
    pca = PCA(n_components=2, whiten=True)
    components = pca.fit_transform(X)
    loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
    xs = components[:,0]
    ys = components[:,1]
    scalex = 1.0/(xs.max() - xs.min())
    scaley = 1.0/(ys.max() - ys.min())
    xl = loadings[:,0]
    yl = loadings[:,1]
    scalex_loading = 1.0/(xl.max() - xl.min())
    scaley_loading = 1.0/(yl.max() - yl.min())
    fig = px.scatter(x=xs * scalex, y=ys * scaley, opacity=0, template="plotly_white")
    for i, feature in enumerate(features):
        fig.add_shape(
            type='line',
            x0=0, y0=0,
            x1=xl[i]* scalex_loading,
            y1=yl[i]* scaley_loading,
            line=dict(color="indigo",
                width=2,
                dash="dot")
        )
        fig.add_annotation(
            x=xl[i]* scalex_loading,
            y=yl[i]* scaley_loading,
            ax=0, ay=0,
            xanchor="center",
            yanchor="bottom",
            text= feature, #"<b>"+feature+"</b>",
            font=dict(
            family="Open Sans, sans serif",
            size=16,
            color="MediumPurple"
            ),
        )
    fig.update_layout(title_text=title, title_x=0.5)
    
    fig.show()

### Rank Correlation of Forecastability Metrics

In [ ]:
def calc_rank(rank_df):
    for col in ['spectral_entropy', 'residual_spectral_entropy',
       'cov',"residual_variability"]:
        rank_df[col] = rank_df[col].rank(ascending=True)

    for col in ['modified_kaboudan_metric',"kaboudan_metric"]:
        rank_df[col] = rank_df[col].rank(ascending=False)

    return rank_df

item_rankings =calc_rank(forecastability_df.drop(columns="LCLid"))
item_rankings.rename(columns=rename_dict, inplace=True)
item_rankings.rename(index=rename_dict, inplace=True)

In [ ]:
corr_df = item_rankings.corr(method='spearman')
# corr_df.style.background_gradient(cmap='coolwarm')

In [ ]:
# mask = np.triu(np.ones_like(corr_df, dtype=bool))
# df_mask = corr_df.mask(mask).round(2)
df_mask = corr_df.round(2)

fig = ff.create_annotated_heatmap(z=df_mask.to_numpy(), 
                                  x=df_mask.columns.tolist(),
                                  y=df_mask.columns.tolist(),
                                  colorscale=px.colors.diverging.RdYlGn,
                                  hoverinfo="none", #Shows hoverinfo for null values
                                  showscale=True, ygap=1, xgap=1
                                 )

fig.update_xaxes(side="bottom")
# fig.update_traces(textfont_size=14)
fig.update_layout(
    title_text='Rank Correlation Plot - Forecastability Metrics', 
    title_x=0.5, 
    width=700, 
    height=700,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    yaxis_autorange='reversed',
    template='plotly_white'
)

# NaN values are not handled automatically and are displayed in the figure
# So we need to get rid of the text manually
for i in range(len(fig.layout.annotations)):
    fig.layout.annotations[i]['font']['size']=15
    if fig.layout.annotations[i].text == 'nan':
        fig.layout.annotations[i].text = ""

# fig.write_image("imgs/chapter_4/rank_correlation_forecastability.png")
fig.show()

### Rank Correlation of Forecastability vs Forecast Metrics

In [ ]:
base_metric_df = pd.read_pickle(output/"baseline_test_metrics_df.pkl")

In [ ]:
base_metric_df.head()

In [ ]:
forecastability_df = forecastability_df.merge(base_metric_df[base_metric_df.Algorithm=="AutoETS"], on="LCLid", how='inner')

In [ ]:
forecastability_df.head()

In [ ]:
def calc_rank(rank_df):
    for col in ['spectral_entropy', 'residual_spectral_entropy',
       'cov',"residual_variability"]:
        rank_df[col] = rank_df[col].rank(ascending=True)

    for col in ['modified_kaboudan_metric',"kaboudan_metric", "mae", "mse", "mase"]:
        rank_df[col] = rank_df[col].rank(ascending=False)
    
    # for col in ["Forecast Bias"]:
    #     rank_df[col] = np.abs(rank_df[col]).rank(ascending=False)

    return rank_df

item_rankings =calc_rank(forecastability_df.drop(columns=["LCLid",'Algorithm']))
item_rankings.rename(columns=rename_dict, inplace=True)
item_rankings.rename(index=rename_dict, inplace=True)
item_rankings.drop(columns=['Time Elapsed', "Model"], inplace=True)
corr_df = item_rankings.corr(method='spearman')

In [ ]:
corr_df.head()

In [ ]:
mask = np.zeros_like(corr_df).astype(bool)
mask[:5,:] = True
df_mask = corr_df.mask(mask).round(2)

fig = ff.create_annotated_heatmap(z=df_mask.to_numpy(), 
                                  x=df_mask.columns.tolist(),
                                  y=df_mask.columns.tolist(),
                                  colorscale=px.colors.diverging.RdYlGn,
                                  hoverinfo="none", #Shows hoverinfo for null values
                                  showscale=True, ygap=1, xgap=1
                                 )

fig.update_xaxes(side="bottom")
# fig.update_traces(textfont_size=14)
fig.update_layout(
    title_text='Rank Correlation Plot - Forecastability vs Forecast Metrics', 
    title_x=0.5, 
    width=700, 
    height=700,
    xaxis_showgrid=False,
    yaxis_showgrid=False,
    xaxis_zeroline=False,
    yaxis_zeroline=False,
    # yaxis_autorange='reversed',
    xaxis_range=[-0.5,5.5],
    yaxis_range=[5.5, 9.5]
)

# NaN values are not handled automatically and are displayed in the figure
# So we need to get rid of the text manually
for i in range(len(fig.layout.annotations)):
    fig.layout.annotations[i]['font']['size']=15
    fig.layout.annotations[i]['font']['color']="#000000"
    if fig.layout.annotations[i].text == 'nan':
        fig.layout.annotations[i].text = ""
# fig.write_image("imgs/chapter_4/rank_correlation_forecastability_vs_metrics.png")
fig.show()

In [ ]:
biplot(item_rankings, features=['Spectral Entropy',"Residual Spectral Entropy",
       'Modified Kaboudan Metric', 'Coefficient of Variation',"Kaboudan Metric",
       'Residual Variability', "mae", "mse", "mase"
       #,"Forecast Bias"
       ], title="<b>Loading Plot: Forecastability Metrics vs Error Metrics</b>")